# Problem statement
To Analyse data inorder to understand and segment customers based on different metrics: recency, frequency and monetary value

# importing Libraries

In [95]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"
import plotly.colors
import datetime as dt

# Loading the dataset

In [2]:
data = pd.read_csv("/content/drive/MyDrive/Datasets/Rfm_dataset.csv")

In [4]:
data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08/11/2021,11/11/2021,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.91
1,2,CA-2016-152156,08/11/2021,11/11/2021,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.58
2,3,CA-2016-138688,12/06/2021,16/06/2021,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.87
3,14,CA-2016-161389,05/12/2021,10/12/2021,Standard Class,IM-15070,Irene Maddox,Consumer,United States,Seattle,...,98103,West,OFF-BI-10003656,Office Supplies,Binders,Fellowes PB200 Plastic Comb Binding Machine,407.98,3,0.2,132.59
4,22,CA-2016-137330,09/12/2021,13/12/2021,Standard Class,KB-16585,Ken Black,Corporate,United States,Fremont,...,68025,Central,OFF-AR-10000246,Office Supplies,Art,Newell 318,19.46,7,0.0,5.06


# Data Exploration

In [6]:
data.shape

(2587, 21)

In [7]:
data.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,2587.000000,2587.000000,2587.000000,2587.000000,2587.000000,2587.000000
mean,4939.622729,54504.607654,235.487383,3.802474,0.154743,31.617793
std,2953.181324,32214.742692,668.217591,2.201917,0.203643,280.780876
min,1.000000,1752.000000,0.840000,1.000000,0.000000,-6599.980000
25%,2233.500000,22153.000000,17.940000,2.000000,0.000000,1.875000
50%,4988.000000,53711.000000,52.680000,3.000000,0.200000,8.740000
75%,7416.500000,90004.000000,208.070000,5.000000,0.200000,28.860000
max,9987.000000,99207.000000,17499.950000,14.000000,0.800000,8399.980000


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2587 entries, 0 to 2586
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         2587 non-null   int64  
 1   Order ID       2587 non-null   object 
 2   Order Date     2587 non-null   object 
 3   Ship Date      2587 non-null   object 
 4   Ship Mode      2587 non-null   object 
 5   Customer ID    2587 non-null   object 
 6   Customer Name  2587 non-null   object 
 7   Segment        2587 non-null   object 
 8   Country        2587 non-null   object 
 9   City           2587 non-null   object 
 10  State          2587 non-null   object 
 11  Postal Code    2587 non-null   int64  
 12  Region         2587 non-null   object 
 13  Product ID     2587 non-null   object 
 14  Category       2587 non-null   object 
 15  Sub-Category   2587 non-null   object 
 16  Product Name   2587 non-null   object 
 17  Sales          2587 non-null   float64
 18  Quantity

In [14]:
missing_values = data.isnull().sum()
print(missing_values)

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64


# Calculating RFM Metrics

In [21]:
data.columns = data.columns.str.strip()

In [24]:
temp = ['Customer ID', 'Order ID', 'Order Date', 'Sales']
RFM_data = data[temp]
RFM_data.shape

(2587, 4)

In [56]:
# convert order date to datetime
RFM_data['Order Date'] = pd.to_datetime(RFM_data['Order Date'], format='%d/%m/%Y')


In [57]:
# calculate the Recency
RFM_data['Recency'] = (RFM_data['Order Date'].max() - RFM_data['Order Date']).dt.days

In [59]:
# calculate the frequency
frequency_data = RFM_data.groupby('Customer ID')['Order ID'].count().reset_index()
frequency_data.rename(columns={'Order ID': 'Frequency'}, inplace=True)
RFM_data = RFM_data.merge(frequency_data, on='Customer ID', how='left')



In [60]:
# Calculate monetary value
monetary_data = RFM_data.groupby('Customer ID')['Sales'].sum().reset_index()
monetary_data.rename(columns={'Sales': 'MonetaryValue'}, inplace=True)
RFM_data = RFM_data.merge(monetary_data, on='Customer ID', how='left')

In [61]:
print(RFM_data.head())

  Customer ID        Order ID Order Date   Sales  Recency  Frequency  \
0    CG-12520  CA-2016-152156 2021-11-08  261.96       53          2   
1    CG-12520  CA-2016-152156 2021-11-08  731.94       53          2   
2    DV-13045  CA-2016-138688 2021-06-12   14.62      202          3   
3    IM-15070  CA-2016-161389 2021-12-05  407.98       26          7   
4    KB-16585  CA-2016-137330 2021-12-09   19.46       22          6   

   MonetaryValue  
0         993.90  
1         993.90  
2          37.60  
3        1108.98  
4         360.03  


# Calculating RFM Scores

In [64]:
# Define scoring criteria
recency_scores = [5, 4, 3, 2, 1] # higher score for lower recency
frequency_scores = [1, 2, 3, 4, 5] # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5] # Higher score for higher monetary value

# Calculate RFM Scores
RFM_data['RecencyScore'] = pd.cut(RFM_data['Recency'], bins=5, labels=recency_scores)
RFM_data['FrequencyScore'] = pd.cut(RFM_data['Frequency'], bins=5, labels=frequency_scores)
RFM_data['MonetaryScore'] = pd.cut(RFM_data['MonetaryValue'], bins=5, labels=monetary_scores)

In [65]:
# Convert the RFM scores to numeric type
RFM_data['RecencyScore'] = RFM_data['RecencyScore'].astype(int)
RFM_data['FrequencyScore'] =RFM_data['FrequencyScore'].astype(int)
RFM_data['MonetaryScore'] =RFM_data['MonetaryScore'].astype(int)

# RFM Segments

In [67]:
RFM_data['RFM_Score'] = RFM_data['RecencyScore'] +  RFM_data['FrequencyScore'] +  RFM_data['MonetaryScore']

# create RFM Segments based on the RFM score
Segment_labels = ['Low-value', 'Mid-value', 'High-Value']
RFM_data['Value Segment'] = pd.qcut(RFM_data['RFM_Score'], q=3, labels=Segment_labels)

In [69]:
print(RFM_data.head())

  Customer ID        Order ID Order Date   Sales  Recency  Frequency  \
0    CG-12520  CA-2016-152156 2021-11-08  261.96       53          2   
1    CG-12520  CA-2016-152156 2021-11-08  731.94       53          2   
2    DV-13045  CA-2016-138688 2021-06-12   14.62      202          3   
3    IM-15070  CA-2016-161389 2021-12-05  407.98       26          7   
4    KB-16585  CA-2016-137330 2021-12-09   19.46       22          6   

   MonetaryValue  RecencyScore  FrequencyScore  MonetaryScore  RFM_Score  \
0         993.90             5               1              1          7   
1         993.90             5               1              1          7   
2          37.60             3               1              1          5   
3        1108.98             5               2              1          8   
4         360.03             5               2              1          8   

  Value Segment  
0     Mid-value  
1     Mid-value  
2     Low-value  
3    High-Value  
4    High-Value  


# Segment distribution

In [85]:
segment_counts = RFM_data['Value Segment'].value_counts().reset_index()
segment_counts.columns = ['Value Segment', 'Count']

fig = px.bar(segment_counts, x='Value Segment', y='Count', color='Value Segment',
             color_discrete_sequence=px.colors.qualitative.Pastel)

fig.show()

# RFM Customer Segments

In [80]:
# Create a new columns for RFM Customer Segments
RFM_data['RFM Customer Segments'] = ''

# Assign RFM segments based on the RFM Score
RFM_data.loc[RFM_data['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
RFM_data.loc[(RFM_data['RFM_Score'] >= 6) & (RFM_data['RFM_Score'] < 9), 'RFM Customer Segments'] = 'High Value'
RFM_data.loc[(RFM_data['RFM_Score'] >= 5) & (RFM_data['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customer'
RFM_data.loc[(RFM_data['RFM_Score'] >= 4) & (RFM_data['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose"
RFM_data.loc[(RFM_data['RFM_Score'] >= 3) & (RFM_data['RFM_Score'] < 4), 'RFM Customer Segments'] = 'Lost'

# Print updated data
print(RFM_data[['Customer ID', 'RFM Customer Segments']])


     Customer ID RFM Customer Segments
0       CG-12520            High Value
1       CG-12520            High Value
2       DV-13045      At Risk Customer
3       IM-15070            High Value
4       KB-16585            High Value
...          ...                   ...
2582    AP-10720             Champions
2583    AP-10720             Champions
2584    RC-19960            High Value
2585    RC-19960            High Value
2586    ML-17410            High Value

[2587 rows x 2 columns]


# RFM Analysis

In [86]:
Segment_product_counts = RFM_data.groupby(['Value Segment', 'RFM Customer Segments']).size().reset_index(name='Count')
Segment_product_counts = Segment_product_counts .sort_values('Count', ascending=False)

# Filter out rows with zero counts
Segment_product_counts = Segment_product_counts[Segment_product_counts['Count'] > 0]

# Create a treemap using Plotly Express
fig_treemap = px.treemap(Segment_product_counts,
                        path=['Value Segment', 'RFM Customer Segments'],
                        values='Count',
                        title='Treemap of Segment Counts',
                        color='Count',
                        color_continuous_scale='Viridis')

# Show the treemap
fig_treemap.show()


In [89]:
# filter the data to include only customers in the champions segment
champions = RFM_data[RFM_data['RFM Customer Segments'] == 'Champions']

fig = go.Figure()
fig.add_trace(go.Box(y=champions['RecencyScore'], name='Recency'))
fig.add_trace(go.Box(y=champions['FrequencyScore'], name='Frequency'))
fig.add_trace(go.Box(y=champions['MonetaryScore'], name='Monetary'))

fig.update_layout(title='Distribution of RFM within Champions segment', yaxis_title='RFM Value', showlegend=True)

fig.show()

In [94]:
corr = champions[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].corr()

fig_heatmap = go.Figure(data=go.Heatmap(z=corr.values,
                                        x=corr.columns,
                                        y=corr.index,
                                        colorscale='cividis',
                                        colorbar=dict(title='Correlation')))
fig_heatmap.update_layout(title='Correlation matrix of RFM within champions segmnent')

fig_heatmap.show()

In [97]:
pastel_colors = plotly.colors.qualitative.Pastel

segment_counts = RFM_data['RFM Customer Segments'].value_counts()

fig = go.Figure(data=go.Bar(x=segment_counts.index,
                            y=segment_counts.values,
                            marker=dict(color=pastel_colors)))

fig.update_layout(title='RFM Customer Segments Bar Chart',
                  xaxis_title='RFM Customer Segments',
                  yaxis_title='Count')

# Show the figure
fig.show()

Create a grouped bar chart to visualize the mean Recency, Frequency, and Monetary scores for each RFM Customer Segment

In [100]:
segment_score = RFM_data.groupby('RFM Customer Segments')['RecencyScore', 'FrequencyScore', 'MonetaryScore'].mean().reset_index()

fig = go.Figure()

# Bars for Recency score
fig.add_trace(go.Bar(
    x=segment_score['RFM Customer Segments'],
    y=segment_score['RecencyScore'],
    name='Recency Score'
))

# Bars for Frequency score
fig.add_trace(go.Bar(
    x=segment_score['RFM Customer Segments'],
    y=segment_score['FrequencyScore'],
    name='Frequency Score'
))

# Bars for Monetary score
fig.add_trace(go.Bar(
    x=segment_score['RFM Customer Segments'],
    y=segment_score['MonetaryScore'],
    name='Monetary Score'
))

# Update layout
fig.update_layout(
    title='Mean RFM Scores by Customer Segment',
    xaxis_title='RFM Customer Segments',
    yaxis_title='Mean Score',
    barmode='group'  # 'group' for grouped bars, 'stack' for stacked bars
)

# Show the figure
fig.show()

<ipython-input-100-289b17dbfb3b>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

